In [1]:
! pip install -U sentence-transformers
! pip install pinecone-client
! pip install faunadb
! pip install ndg-httpsclient
! pip install pyopenssl
! pip install pyasn1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=da336559ab59c0a9a06c8858eac7422aeafdc9e45154bd3bf147acc6c7878710
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.9 MB/s



*   AB = Column that corresponds to the text of the abstract
*   batch_input = List that will be populated with text-id pairs, each abstract gets an id so that it can be retrieved from the DB

In [3]:
import csv
from itertools import islice

# Load data, store abstract text
batch_input = []
with open('/content/drive/MyDrive/data/INLPT_data/articles.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        batch_input.append([row["AB"]])

PubMedBert is a on pubmed data finetuned BERT sentence embedding model. We embedd per Abstract


*   https://huggingface.co/pritamdeka/S-PubMedBert-MS-MARCO




In [ ]:
from sentence_transformers import SentenceTransformer

#embed data
model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
embeddings = [model.encode(chunk[0]) for chunk in batch_input]

Create a vector that holds the data that is going to be uploaded to Pinecone Vector DB. To Pinecone we upload the embedding together with an id that maps the embedding to its original text in the DB (FaunaDB)

In [ ]:
import uuid

vectors = []

#prepare data for upload to vectorDB
for element in batch_input:
  element.append(str(uuid.uuid4()))

createvec = [{vectors.append((
    data[1], #id of paragraph
    embeddings[i].tolist() #paragraph embedding
    ))} for i, data in enumerate(batch_input)],

In [ ]:
import pinecone

#Init pinecone index
pinecone.init(api_key="4d2c2cd0-cf55-43c5-afb1-11001dc68709", environment="gcp-starter")
index = pinecone.Index("inlp-med-ws2324")


In [ ]:
import random
import itertools

#Split data into m n-sized lists, used for bulk upload
def chunks(iterable, n):
  chunks = [iterable[x:x+n] for x in range(0, len(iterable), n)]
  return chunks


In [ ]:
#upsert data to pinecone
chuk = chunks(vectors)
for ids_vectors_chunk in chunks(vectors):
  index.upsert(vectors=ids_vectors_chunk)

Here we add a new column to "id" to the dataframe, later we upload each row of the file as a record into FaunaDB


*   https://v4.dashboard.fauna.com/db/eu/medicalData




In [ ]:
import pandas as pd

def add_id_column(file_name, vectors):

    list_with_id = []
    for data in vectors:
      list_with_id.append(data[0])

    df = pd.read_csv(file_name)
    df['id'] = list_with_id
    df.to_csv(file_name, index=False)

In [ ]:
add_id_column('/content/drive/MyDrive/data/INLPT_data/articles.csv', vectors)

In [ ]:
import pandas as pd
#showcase data
df = pd.read_csv('/content/drive/MyDrive/data/INLPT_data/articles.csv')
df

,PMID,TI,AB,PB,FAU,FED,DP,OTO,OT,OWN,DCOM,LR,JT,MH,ISBN,id
0,24278995,CASK Disorders.,CLINICAL CHARACTERISTICS: CASK disorders inclu...,"University of Washington, Seattle","Moog, Ute|Kutsche, Kerstin","Adam, Margaret P|Feldman, Jerry|Mirzaa, Ghayda...",1993,NLM,Intellectual Disability and Microcephaly with ...,NaN,NaN,NaN,NaN,NaN,NaN,fedf5381-d8ab-4c53-b2ed-360b0d5bc853
1,25529590,WAIS-IV administration errors: effects of alte...,This study utilized a sample of 50 college stu...,NaN,"Ryan, Joseph J|Swopes-Willhite, Nicole|Frankli...",NaN,2015,NOTNLM,Block Design|Symbol Search|WAIS-IV|intelligence,NLM,20151224.0,20141222.0,Applied neuropsychology. Adult,Adolescent|Adult|Female|Humans|*Intelligence|M...,NaN,9ddabfa8-b644-4e42-bf2f-7a20697a2788
2,25529585,Apparently abnormal Wechsler Memory Scale inde...,Interpretation of the Wechsler Memory Scale-Fo...,NaN,"Carrasco, Roman Marcus|Grups, Josefine|Evans, ...",NaN,2015,NOTNLM,General Ability Index|WAIS-IV|WMS-IV,NLM,20151224.0,20141222.0,Applied neuropsychology. Adult,Humans|Memory/*physiology|Monte Carlo Method|*...,NaN,8132fa46-1eb3-4e64-bfe2-cfaf2c0a1ee4
3,25284715,A longitudinal intergenerational analysis of e...,Despite the importance of executive function (...,NaN,"Cuevas, Kimberly|Deater-Deckard, Kirby|Kim-Spo...",NaN,2014 Mar,NOTNLM,early childhood|executive function|individual ...,NLM,20150610.0,20211021.0,The British journal of developmental psychology,"Adult|*Child Development|Child, Preschool|*Exe...",NaN,3d98b075-bf2b-49bb-9c17-863a236a3bc5
4,25265311,The design organization test: further demonstr...,Neuropsychological assessments are frequently ...,NaN,"Killgore, William D S|Gogel, Hannah",NaN,2014,NOTNLM,IQ|neuropsychology|reliability|validity|visuos...,NLM,20150715.0,20211021.0,Applied neuropsychology. Adult,Adolescent|Adult|Female|Humans|Intelligence/*p...,NaN,f6a45fb2-86ee-4a38-9bd1-3f4012e161e5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57555,35294355,A Survey of Hardware Self-Organizing Maps.,Self-organizing feature maps (SOMs) are common...,NaN,"Jovanovic, Slavisa|Hikawa, Hiroomi",NaN,2023 Nov,NaN,NaN,NLM,NaN,20231028.0,IEEE transactions on neural networks and learn...,NaN,NaN,7ee3c5de-ec06-4e22-ac28-59241bcae7db
57556,35235526,Efficient Architecture Search for Continual Le...,"Continual learning with neural networks, which...",NaN,"Gao, Qiang|Luo, Zhipeng|Klabjan, Diego|Zhang, ...",NaN,2023 Nov,NaN,NaN,NLM,NaN,20231028.0,IEEE transactions on neural networks and learn...,NaN,NaN,f9ccc757-72a2-4612-b525-1dc2fd2f8400
57557,35230953,QTT-DLSTM: A Cloud-Edge-Aided Distributed LSTM...,"Cyber-physical-social systems (CPSS), an emerg...",NaN,"Wang, Xiaokang|Ren, Lei|Yuan, Ruixue|Yang, Lau...",NaN,2023 Oct,NaN,NaN,NLM,NaN,20231020.0,IEEE transactions on neural networks and learn...,NaN,NaN,ea4278d7-fc28-44b3-9879-994486f6d23f
57558,35130174,A Review of Recurrent Neural Network-Based Met...,Artificial intelligence and machine learning t...,NaN,"Mao, Shitong|Sejdic, Ervin",NaN,2023 Oct,NaN,NaN,NLM,20231006.0,20231023.0,IEEE transactions on neural networks and learn...,"Humans|*Neural Networks, Computer|*Artificial ...",NaN,b37db766-d99a-4a46-bf0d-b187342663c8


In [ ]:
#Initialize FaunaDB
from faunadb import query as q
from faunadb.objects import Ref
from faunadb.client import FaunaClient

client = FaunaClient(
  secret="fnAFU2WrkwAAzGaOBrMY__25MeF1irzQ39nHM9J6",
)

In [ ]:
reader = csv.reader(open('/content/drive/MyDrive/data/INLPT_data/articles.csv'), delimiter=';')
with open('/content/drive/MyDrive/data/INLPT_data/articles.csv') as f:
  lines = list(csv.reader(f))

lines.pop(0) #Pop the line that only stores column names

chunkeddata = chunks(lines, 5000)

In [ ]:
#Upload data into FaunaDB
def upload_data_to_fauna(data):
        for row in data:
            client.query(q.create(q.collection("metadata"),{"data": {"PMID": row[0], "TI": row[1],"AB": row[2],"PB": row[3],"FAU": row[4],"FED": row[5],"DP": row[6],"OTO": row[7],"OT": row[8],"OWN": row[9],"DCOM": row[10],"LR": row[11],"JT": row[12],"MH": row[13],"ISBN": row[14], "id": row[15]}}))

In [ ]:
upload_data_to_fauna(chunkeddata[0])

How to query from Pinecone

In [ ]:
def return_document(query):
  embeded_vector = model.encode(query)

  query_response = index.query(
      embeded_vector,
      top_k=3,
      include_metadata=True)
  return query_response

return_document("Medical Data about Birth control on females")

How to Query from FaunaDB

In [ ]:
result = client.query(
  q.create_index({
    "name": "abstractid",
    "source": q.collection("abstracts")
  })
)
print(result)

{'ref': Ref(id=abstractid, collection=Ref(id=indexes)), 'ts': 1702246281230000, 'active': True, 'serialized': True, 'name': 'abstractid', 'source': Ref(id=abstracts, collection=Ref(id=collections)), 'partitions': 8}


ToDo: Implement Pinecone and FaunaDB API into backend
